In [ ]:
#!pip3 install torch torchvision torchaudio

In [ ]:
#!python3 -m pip install tensorflow

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import random
import matplotlib.pyplot as plt
import seaborn as sns
import datetime


from sklearn.metrics import roc_auc_score

import pickle
def dump_pkl(data, filename):
  with open(filename, 'wb') as handle:
    pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

def load_pkl(filename):
  with open(filename, 'rb') as handle:
    data = pickle.load(handle)
  return data

### 1. Load data

In [3]:
dfs_1min = []
dfs_5min = []
dfs_1hour = []
dfs_1day = []

stocks = [
          'GAZP',
          'SBER',
          'LKOH',
          'MGNT',
          'NVTK',
          'SNGS', 'SNGSP',
          'GMKN',
          'ROSN',
          'NLMK',
          'TATN',
          'MTSS',
          'ALRS',  
          'YDEX',
          'CHMF',
          
          'MAGN',
          'T',
          'OZON',
          'RUAL'  
         ]
for stock in tqdm(stocks):
    df_1min = load_pkl(f"./data/preproc/1min/{stock}.pkl")
    df_5min = load_pkl(f"./data/preproc/5min/{stock}.pkl")
    df_1hour = load_pkl(f"./data/preproc/1hour/{stock}.pkl")
    df_1day = load_pkl(f"./data/preproc/1day/{stock}.pkl")
    
    dfs_1min += [df_1min.copy()]
    dfs_5min += [df_5min.copy()]
    dfs_1hour += [df_1hour.copy()]
    dfs_1day += [df_1day.copy()]
    



df_1min = pd.concat(dfs_1min)
df_5min = pd.concat(dfs_5min)
df_1day = pd.concat(dfs_1day)
df_1hour = pd.concat(dfs_1hour)

df_1min.reset_index(drop=True, inplace=True)
df_5min.reset_index(drop=True, inplace=True)
df_1hour.reset_index(drop=True, inplace=True)
df_1day.reset_index(drop=True, inplace=True)

df_1min.shape, df_5min.shape, df_1hour.shape, df_1day.shape

100%|█████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 53.39it/s]


((8666393, 7), (1777831, 7), (152890, 7), (10761, 7))

In [6]:
df_1min.head()

,time,open,close,volume,low,high,ticker
0,2022-10-03 10:00:00,219.00,219.93,87793.0,218.00,220.25,GAZP
1,2022-10-03 10:01:00,219.80,220.08,61549.0,219.07,220.55,GAZP
2,2022-10-03 10:02:00,220.24,219.34,52640.0,219.02,220.38,GAZP
3,2022-10-03 10:03:00,219.17,218.96,61506.0,217.70,219.40,GAZP
4,2022-10-03 10:04:00,218.82,218.30,51464.0,217.76,218.96,GAZP


### 2. Preproc data

#### 2.1 Make target

In [15]:
# Используем уже знакомую функцию :)

def get_target(df_all, ind, val_first, val_second, hours_to_wait):
    #цель - достичь val_second, не достигнув val_first

    points_in_hour = 60 # час
    points_to_wait = hours_to_wait * points_in_hour
    ind_end = min(ind+points_to_wait+1, df_all.shape[0])
    
    df = df_all.iloc[ind:ind_end].copy()
    
    mask_stock = np.array(df['ticker'] == df['ticker'].iloc[0])
    df = df.loc[mask_stock, :]
    
    if val_first < val_second:                                 
        mask_val_first = np.array(df['close'] < val_first)     #### ХОТЯ БЫ ТАК, уже с 20 до 18 опусает WinRate (смена close -> low). Обратно откатил
        mask_val_second = np.array(df['close'] > val_second)
    else: 
        mask_val_first = np.array(df['close'] > val_first)      ### тут будущему мне поразбираться нужно будет
        mask_val_second = np.array(df['close'] < val_second)

    
    if (mask_val_first.sum() == 0) & (mask_val_second.sum() == 0):
        return 'DNF', -1, -1, -1
        
    if (mask_val_first.sum() == 0) & (mask_val_second.sum() != 0):
        ind_val_second = np.argwhere(mask_val_second).ravel()[0]
        delta_time = df['time'].iloc[ind_val_second] - df['time'].iloc[0]
        res_price = df['close'].iloc[ind_val_second]
        return 'WIN', delta_time, res_price, ind_val_second+ind
        
    if (mask_val_first.sum() != 0) & (mask_val_second.sum() == 0):
        ind_val_first = np.argwhere(mask_val_first).ravel()[0]
        delta_time = df['time'].iloc[ind_val_first] - df['time'].iloc[0]
        res_price = df['close'].iloc[ind_val_first]
        return 'LOSE', delta_time, res_price, ind_val_first+ind

    if (mask_val_first.sum() != 0) & (mask_val_second.sum() != 0):
        ind_val_first = np.argwhere(mask_val_first).ravel()[0]
        ind_val_second = np.argwhere(mask_val_second).ravel()[0]
        if ind_val_first < ind_val_second:
            delta_time = df['time'].iloc[ind_val_first] - df['time'].iloc[0]
            res_price = df['close'].iloc[ind_val_first]
            return 'LOSE', delta_time, res_price, ind_val_first+ind
        if ind_val_first > ind_val_second:
            delta_time = df['time'].iloc[ind_val_second] - df['time'].iloc[0]
            res_price = df['close'].iloc[ind_val_second]
            return 'WIN', delta_time, res_price, ind_val_second+ind


def get_df_target(df, indx, percent_first=None, percent_second=None, hours_to_wait=None):
    times = []
    results = []
    delta_times = []
    closes = []
    tickers = []
    res_prices = []
    res_inds = []
    
    #for ind in indx:
    for ind in tqdm(indx):
        time = df['time'].iloc[ind]
        close = df['close'].iloc[ind]
        ticker = df['ticker'].iloc[ind]
        
        val_first = df['close'].iloc[ind] * percent_first
        val_second = df['close'].iloc[ind] * percent_second
        result, delta_time, res_price, res_ind = get_target(df, ind, val_first, val_second, hours_to_wait)
        
        times += [time]
        closes += [close]
        tickers += [ticker]
        results += [result]
        delta_times += [delta_time]
        res_prices += [res_price]
        res_inds += [res_ind]
        

    df_result = pd.DataFrame({'ind' : indx,
                              'time' : times,
                              'close' : closes,
                              'result' : results,
                              'ticker' : tickers, 
                              'delta_time' : delta_times,
                              'res_price' : res_prices,
                              'res_ind' : res_inds
                             })
    return df_result    

### start_TMP

In [101]:
# Поимка бага :)
df_result = get_df_target(df_1min, np.arange(2744364, 2744389), percent_first=0.995, percent_second=1.015, hours_to_wait=14)
df_result

100%|████████████████████████████████████████████████████████████████████████| 25/25 [00:00<00:00, 909.62it/s]


,ind,time,close,result,ticker,delta_time,res_price,res_ind
0,2744364,2024-12-13 13:33:00,22.675,LOSE,SNGS,2 days 20:27:00,22.700,2744853
1,2744365,2024-12-13 13:34:00,22.670,LOSE,SNGS,2 days 20:26:00,22.700,2744853
2,2744366,2024-12-13 13:35:00,22.715,LOSE,SNGS,0 days 01:26:00,22.590,2744444
3,2744367,2024-12-13 13:36:00,22.685,LOSE,SNGS,2 days 20:24:00,22.700,2744853
4,2744368,2024-12-13 13:37:00,22.680,LOSE,SNGS,2 days 20:23:00,22.700,2744853
5,2744369,2024-12-13 13:39:00,22.685,LOSE,SNGS,2 days 20:21:00,22.700,2744853
6,2744370,2024-12-13 13:40:00,22.675,LOSE,SNGS,2 days 20:20:00,22.700,2744853
7,2744371,2024-12-13 13:41:00,22.675,LOSE,SNGS,2 days 20:19:00,22.700,2744853
8,2744372,2024-12-13 13:42:00,22.680,LOSE,SNGS,2 days 20:18:00,22.700,2744853
9,2744373,2024-12-13 13:43:00,22.670,LOSE,SNGS,2 days 20:17:00,22.700,2744853


### end_TMP

In [20]:
inds = np.arange(df_1min.shape[0])
inds.shape

(8666393,)

In [24]:
df_result = get_df_target(df_1min, inds, percent_first=0.995, percent_second=1.005, hours_to_wait=1)

100%|█████████████████████████████████████████████████████████████| 8666393/8666393 [29:55<00:00, 4826.91it/s]


In [25]:
df_result

,ind,time,close,result,ticker,delta_time,res_price,res_ind
0,0,2022-10-03 10:00:00,219.930,LOSE,GAZP,0 days 00:04:00,218.30,4
1,1,2022-10-03 10:01:00,220.080,LOSE,GAZP,0 days 00:02:00,218.96,3
2,2,2022-10-03 10:02:00,219.340,LOSE,GAZP,0 days 00:03:00,217.24,5
3,3,2022-10-03 10:03:00,218.960,LOSE,GAZP,0 days 00:02:00,217.24,5
4,4,2022-10-03 10:04:00,218.300,LOSE,GAZP,0 days 00:07:00,216.80,11
...,...,...,...,...,...,...,...,...
8666388,8666388,2024-12-25 23:45:00,32.690,DNF,RUAL,-1,-1.00,-1
8666389,8666389,2024-12-25 23:46:00,32.650,DNF,RUAL,-1,-1.00,-1
8666390,8666390,2024-12-25 23:47:00,32.665,DNF,RUAL,-1,-1.00,-1
8666391,8666391,2024-12-25 23:48:00,32.705,DNF,RUAL,-1,-1.00,-1


In [27]:
df_result['result'].value_counts(normalize=True)

result
DNF     0.671223
LOSE    0.165617
WIN     0.163160
Name: proportion, dtype: float64

In [28]:
(df_result['ind'] == df_result.index).all(), 

(True,)

In [32]:
!mkdir experiments

mkdir: experiments: File exists


In [33]:
!mkdir experiments/feat_engin

mkdir: experiments/feat_engin: File exists


In [34]:
dump_pkl(df_result, './experiments/feat_engin/df_result_wait_1hour.pkl')

#### 2.1.2 Загрузим датасет с ожиданием в 1 день

In [3]:
df_result = load_pkl('./experiments/feat_engin/df_result_wait_1hour.pkl')

In [4]:
df_result

,ind,time,close,result,ticker,delta_time,res_price,res_ind
0,0,2022-10-03 10:00:00,219.930,LOSE,GAZP,0 days 00:04:00,218.30,4
1,1,2022-10-03 10:01:00,220.080,LOSE,GAZP,0 days 00:02:00,218.96,3
2,2,2022-10-03 10:02:00,219.340,LOSE,GAZP,0 days 00:03:00,217.24,5
3,3,2022-10-03 10:03:00,218.960,LOSE,GAZP,0 days 00:02:00,217.24,5
4,4,2022-10-03 10:04:00,218.300,LOSE,GAZP,0 days 00:07:00,216.80,11
...,...,...,...,...,...,...,...,...
8666388,8666388,2024-12-25 23:45:00,32.690,DNF,RUAL,-1,-1.00,-1
8666389,8666389,2024-12-25 23:46:00,32.650,DNF,RUAL,-1,-1.00,-1
8666390,8666390,2024-12-25 23:47:00,32.665,DNF,RUAL,-1,-1.00,-1
8666391,8666391,2024-12-25 23:48:00,32.705,DNF,RUAL,-1,-1.00,-1


In [5]:
df_result['delta_time'] = df_result['delta_time'].replace(-1, pd.Timedelta('100 days'))
df_result

/var/folders/rm/zt5_90s5727289mv2gzbkw980000gn/T/ipykernel_21398/3638154976.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_result['delta_time'] = df_result['delta_time'].replace(-1, pd.Timedelta('100 days'))


,ind,time,close,result,ticker,delta_time,res_price,res_ind
0,0,2022-10-03 10:00:00,219.930,LOSE,GAZP,0 days 00:04:00,218.30,4
1,1,2022-10-03 10:01:00,220.080,LOSE,GAZP,0 days 00:02:00,218.96,3
2,2,2022-10-03 10:02:00,219.340,LOSE,GAZP,0 days 00:03:00,217.24,5
3,3,2022-10-03 10:03:00,218.960,LOSE,GAZP,0 days 00:02:00,217.24,5
4,4,2022-10-03 10:04:00,218.300,LOSE,GAZP,0 days 00:07:00,216.80,11
...,...,...,...,...,...,...,...,...
8666388,8666388,2024-12-25 23:45:00,32.690,DNF,RUAL,100 days 00:00:00,-1.00,-1
8666389,8666389,2024-12-25 23:46:00,32.650,DNF,RUAL,100 days 00:00:00,-1.00,-1
8666390,8666390,2024-12-25 23:47:00,32.665,DNF,RUAL,100 days 00:00:00,-1.00,-1
8666391,8666391,2024-12-25 23:48:00,32.705,DNF,RUAL,100 days 00:00:00,-1.00,-1


In [6]:
df_result['result'].value_counts(normalize=True)

result
DNF     0.671223
LOSE    0.165617
WIN     0.163160
Name: proportion, dtype: float64

#### 2.2 Link data of different time-period

In [10]:
dfs_1min = []
dfs_5min = []
dfs_1hour = []
dfs_1day = []

stocks = [
          'GAZP',
          'SBER',
          'LKOH',
          'MGNT',
          'NVTK',
          'SNGS', 'SNGSP',
          'GMKN',
          'ROSN',
          'NLMK',
          'TATN',
          'MTSS',
          'ALRS',  
          'YDEX',
          'CHMF',
          
          'MAGN',
          'T',
          'OZON',
          'RUAL'  
         ]
for stock in tqdm(stocks):
    df_1min = load_pkl(f"./data/preproc/1min/{stock}.pkl")
    df_5min = load_pkl(f"./data/preproc/5min/{stock}.pkl")
    df_1hour = load_pkl(f"./data/preproc/1hour/{stock}.pkl")
    df_1day = load_pkl(f"./data/preproc/1day/{stock}.pkl")

    df_1min['index_1min_start'] = None
    df_1min.loc[0, 'index_1min_start'] = 42
    df_5min['index_start'] = None
    df_5min.loc[0, 'index_start'] = 42
    df_1hour['index_start'] = None
    df_1hour.loc[0, 'index_start'] = 42
    df_1day['index_start'] = None
    df_1day.loc[0, 'index_start'] = 42

    dfs_1min += [df_1min.copy()]
    dfs_5min += [df_5min.copy()]
    dfs_1hour += [df_1hour.copy()]
    dfs_1day += [df_1day.copy()]
    



df_1min = pd.concat(dfs_1min)
df_5min = pd.concat(dfs_5min)
df_1day = pd.concat(dfs_1day)
df_1hour = pd.concat(dfs_1hour)

df_1min.reset_index(drop=True, inplace=True)
df_5min.reset_index(drop=True, inplace=True)
df_1hour.reset_index(drop=True, inplace=True)
df_1day.reset_index(drop=True, inplace=True)

df_1min.shape, df_5min.shape, df_1hour.shape, df_1day.shape

100%|█████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 31.18it/s]


((8666393, 8), (1777831, 8), (152890, 8), (10761, 8))

In [11]:
pd.set_option('future.no_silent_downcasting', True)

#start index in each df-time
#1min
mask = df_1min['index_1min_start'] == 42
df_1min.loc[mask, 'index_1min_start'] = df_1min.index[mask]
df_1min['index_1min_start'] = df_1min['index_1min_start'].ffill()

#5min
mask = df_5min['index_start'] == 42
df_5min.loc[mask, 'index_start'] = df_5min.index[mask]
df_5min['index_start'] = df_5min['index_start'].ffill()

#1hour
mask = df_1hour['index_start'] == 42
df_1hour.loc[mask, 'index_start'] = df_1hour.index[mask]
df_1hour['index_start'] = df_1hour['index_start'].ffill()

#1day
mask = df_1day['index_start'] == 42
df_1day.loc[mask, 'index_start'] = df_1day.index[mask]
df_1day['index_start'] = df_1day['index_start'].ffill()

In [12]:
#link different time-step date

df_5min = df_5min.reset_index()
df_1hour = df_1hour.reset_index()
df_1day = df_1day.reset_index()

df_5min = df_5min.rename(columns={col : col+'_5min' for col in df_5min.columns if col not in ['time', 'ticker']})
df_1hour = df_1hour.rename(columns={col : col+'_1hour' for col in df_1hour.columns if col not in ['time', 'ticker']})
df_1day = df_1day.rename(columns={col : col+'_1day' for col in df_1day.columns if col not in ['time', 'ticker']})

df_1min['time'] += pd.Timedelta('1min')
df_5min['time'] += pd.Timedelta('5min')
df_1hour['time'] += pd.Timedelta('1hour')
#1) биржа работает до 23:50 
mask = datetime.time(0, 0) == pd.to_datetime(df_1hour['time'], format='%H:%M').dt.time
df_1hour.loc[mask, 'time'] -= pd.Timedelta('10min')
#2)и клирингс 18:50 до 19:05
mask = datetime.time(19, 0) == pd.to_datetime(df_1hour['time'], format='%H:%M').dt.time
df_1hour.loc[mask, 'time'] -= pd.Timedelta('10min')
# end
df_1day['time'] += pd.Timedelta('20:50:00')


df = df_1min.merge(df_5min, on=['time', 'ticker'], how='left')
df = df.merge(df_1hour, on=['time', 'ticker'], how='left')
df = df.merge(df_1day, on=['time', 'ticker'], how='left')

#Проверка, что данные все данные подтянулись
print(f'Пересесчение в данных 1min и 5min: {df['close_5min'].notnull().sum() / df_5min.shape[0]}')
print(f'Пересесчение в данных 1min и 1hour: {df['close_1hour'].notnull().sum() / df_1hour.shape[0]}')
print(f'Пересесчение в данных 1min и 1day: {df['close_1day'].notnull().sum() / df_1day.shape[0]}')
#в минутных данных бывают пропуски, это можно проинтерполировать. Но пока просто игнорируем, вроде как несущественный эффект должен быть

#Проверка на равенство значений
count_diff = ((df['close_5min'].notnull()) & (df['close'] != df['close_5min'])).sum()
assert count_diff == 0, f'Error: close & close_5min, count diff values: {count_diff}' 
count_diff = ((df['close_1hour'].notnull()) & (df['close'] != df['close_1hour'])).sum()
assert count_diff < 200, f'Error: close & close_1hour, count diff values: {count_diff}' #тут есть празничные торги они идут без вечернего клиринга и без вечерних торгов
count_diff = ((df['close_1day'].notnull()) & (df['close'] != df['close_1day'])).sum()
assert count_diff == 0, f'Error: close & close_1day, count diff values: {count_diff}' 

#Проверка что не наджоинилось лишнего
assert df_1min.shape[0] == df.shape[0], 'Error: with join dimensions'

Пересесчение в данных 1min и 5min: 0.9792657457317372
Пересесчение в данных 1min и 1hour: 0.9656746680620054
Пересесчение в данных 1min и 1day: 0.9886627636836726


In [13]:
for stock in tqdm(stocks):
    mask_stock = df['ticker'] == stock
    
    cols_ffill = ['index_1min_start', 'index_start_5min', 'index_5min', 'index_start_1hour', 'index_1hour', 'index_start_1day', 'index_1day']
    df.loc[mask_stock, cols_ffill] = df.loc[mask_stock, cols_ffill].ffill()

100%|█████████████████████████████████████████████████████████████████████████| 19/19 [00:12<00:00,  1.55it/s]


In [14]:
df

,time,open,close,volume,low,high,ticker,index_1min_start,index_5min,open_5min,...,low_1hour,high_1hour,index_start_1hour,index_1day,open_1day,close_1day,volume_1day,low_1day,high_1day,index_start_1day
0,2022-10-03 10:01:00,219.000,219.930,87793.0,218.000,220.250,GAZP,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-10-03 10:02:00,219.800,220.080,61549.0,219.070,220.550,GAZP,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-10-03 10:03:00,220.240,219.340,52640.0,219.020,220.380,GAZP,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-10-03 10:04:00,219.170,218.960,61506.0,217.700,219.400,GAZP,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-10-03 10:05:00,218.820,218.300,51464.0,217.760,218.960,GAZP,0,0.0,219.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8666388,2024-12-25 23:46:00,32.690,32.690,1.0,32.690,32.690,RUAL,8208890,1777829.0,NaN,...,NaN,NaN,144800,10759.0,NaN,NaN,NaN,NaN,NaN,10192
8666389,2024-12-25 23:47:00,32.670,32.650,123.0,32.650,32.670,RUAL,8208890,1777829.0,NaN,...,NaN,NaN,144800,10759.0,NaN,NaN,NaN,NaN,NaN,10192
8666390,2024-12-25 23:48:00,32.650,32.665,169.0,32.645,32.665,RUAL,8208890,1777829.0,NaN,...,NaN,NaN,144800,10759.0,NaN,NaN,NaN,NaN,NaN,10192
8666391,2024-12-25 23:49:00,32.680,32.705,2003.0,32.675,32.705,RUAL,8208890,1777829.0,NaN,...,NaN,NaN,144800,10759.0,NaN,NaN,NaN,NaN,NaN,10192


#### 2.4 Union target and features. Make file to train

In [17]:
(df_result['time']+pd.Timedelta('1min') == df['time']).all(), (df_result['close'] == df['close']).all()

(True, True)

In [18]:
(df.index == df_result.index).all()

True

In [19]:
df_result.head()

,ind,time,close,result,ticker,delta_time,res_price,res_ind
0,0,2022-10-03 10:00:00,219.93,LOSE,GAZP,0 days 00:04:00,218.30,4
1,1,2022-10-03 10:01:00,220.08,LOSE,GAZP,0 days 00:02:00,218.96,3
2,2,2022-10-03 10:02:00,219.34,LOSE,GAZP,0 days 00:03:00,217.24,5
3,3,2022-10-03 10:03:00,218.96,LOSE,GAZP,0 days 00:02:00,217.24,5
4,4,2022-10-03 10:04:00,218.30,LOSE,GAZP,0 days 00:07:00,216.80,11


In [21]:
#union
df = pd.concat([df, df_result[['result', 'delta_time', 'res_price', 'res_ind']]], axis=1)
df

,time,open,close,volume,low,high,ticker,index_1min_start,index_5min,open_5min,...,open_1day,close_1day,volume_1day,low_1day,high_1day,index_start_1day,result,delta_time,res_price,res_ind
0,2022-10-03 10:01:00,219.000,219.930,87793.0,218.000,220.250,GAZP,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,LOSE,0 days 00:04:00,218.30,4
1,2022-10-03 10:02:00,219.800,220.080,61549.0,219.070,220.550,GAZP,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,LOSE,0 days 00:02:00,218.96,3
2,2022-10-03 10:03:00,220.240,219.340,52640.0,219.020,220.380,GAZP,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,LOSE,0 days 00:03:00,217.24,5
3,2022-10-03 10:04:00,219.170,218.960,61506.0,217.700,219.400,GAZP,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,LOSE,0 days 00:02:00,217.24,5
4,2022-10-03 10:05:00,218.820,218.300,51464.0,217.760,218.960,GAZP,0,0.0,219.00,...,NaN,NaN,NaN,NaN,NaN,NaN,LOSE,0 days 00:07:00,216.80,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8666388,2024-12-25 23:46:00,32.690,32.690,1.0,32.690,32.690,RUAL,8208890,1777829.0,NaN,...,NaN,NaN,NaN,NaN,NaN,10192,DNF,100 days 00:00:00,-1.00,-1
8666389,2024-12-25 23:47:00,32.670,32.650,123.0,32.650,32.670,RUAL,8208890,1777829.0,NaN,...,NaN,NaN,NaN,NaN,NaN,10192,DNF,100 days 00:00:00,-1.00,-1
8666390,2024-12-25 23:48:00,32.650,32.665,169.0,32.645,32.665,RUAL,8208890,1777829.0,NaN,...,NaN,NaN,NaN,NaN,NaN,10192,DNF,100 days 00:00:00,-1.00,-1
8666391,2024-12-25 23:49:00,32.680,32.705,2003.0,32.675,32.705,RUAL,8208890,1777829.0,NaN,...,NaN,NaN,NaN,NaN,NaN,10192,DNF,100 days 00:00:00,-1.00,-1


### 2.5 Feature engineering

In [28]:
df

,time,open,close,volume,low,high,ticker,index_1min_start,index_5min,open_5min,...,open_1day,close_1day,volume_1day,low_1day,high_1day,index_start_1day,result,delta_time,res_price,res_ind
0,2022-10-03 10:01:00,219.000,219.930,87793.0,218.000,220.250,GAZP,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,LOSE,0 days 00:04:00,218.30,4
1,2022-10-03 10:02:00,219.800,220.080,61549.0,219.070,220.550,GAZP,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,LOSE,0 days 00:02:00,218.96,3
2,2022-10-03 10:03:00,220.240,219.340,52640.0,219.020,220.380,GAZP,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,LOSE,0 days 00:03:00,217.24,5
3,2022-10-03 10:04:00,219.170,218.960,61506.0,217.700,219.400,GAZP,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,LOSE,0 days 00:02:00,217.24,5
4,2022-10-03 10:05:00,218.820,218.300,51464.0,217.760,218.960,GAZP,0,0.0,219.00,...,NaN,NaN,NaN,NaN,NaN,NaN,LOSE,0 days 00:07:00,216.80,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8666388,2024-12-25 23:46:00,32.690,32.690,1.0,32.690,32.690,RUAL,8208890,1777829.0,NaN,...,NaN,NaN,NaN,NaN,NaN,10192,DNF,100 days 00:00:00,-1.00,-1
8666389,2024-12-25 23:47:00,32.670,32.650,123.0,32.650,32.670,RUAL,8208890,1777829.0,NaN,...,NaN,NaN,NaN,NaN,NaN,10192,DNF,100 days 00:00:00,-1.00,-1
8666390,2024-12-25 23:48:00,32.650,32.665,169.0,32.645,32.665,RUAL,8208890,1777829.0,NaN,...,NaN,NaN,NaN,NaN,NaN,10192,DNF,100 days 00:00:00,-1.00,-1
8666391,2024-12-25 23:49:00,32.680,32.705,2003.0,32.675,32.705,RUAL,8208890,1777829.0,NaN,...,NaN,NaN,NaN,NaN,NaN,10192,DNF,100 days 00:00:00,-1.00,-1


In [29]:
df_5min.head()

,index_5min,time,open_5min,close_5min,volume_5min,low_5min,high_5min,ticker,index_start_5min
0,0,2022-10-03 10:05:00,219.00,218.30,314952.0,217.70,220.55,GAZP,0
1,1,2022-10-03 10:10:00,218.30,218.05,182525.0,216.19,218.78,GAZP,0
2,2,2022-10-03 10:15:00,218.11,215.86,193758.0,215.31,218.13,GAZP,0
3,3,2022-10-03 10:20:00,215.83,216.85,91747.0,215.70,217.39,GAZP,0
4,4,2022-10-03 10:25:00,216.85,216.58,52824.0,216.40,217.16,GAZP,0


In [30]:
df_1hour.head()

,index_1hour,time,open_1hour,close_1hour,volume_1hour,low_1hour,high_1hour,ticker,index_start_1hour
0,0,2022-10-03 11:00:00,219.00,215.08,1623411.0,213.63,220.55,GAZP,0
1,1,2022-10-03 12:00:00,215.08,216.36,411786.0,214.96,216.78,GAZP,0
2,2,2022-10-03 13:00:00,216.34,215.88,237084.0,215.43,216.43,GAZP,0
3,3,2022-10-03 14:00:00,215.85,217.10,573509.0,215.53,217.77,GAZP,0
4,4,2022-10-03 15:00:00,217.12,216.94,364609.0,216.50,217.49,GAZP,0


In [31]:
df_1day.head()

,index_1day,time,open_1day,close_1day,volume_1day,low_1day,high_1day,ticker,index_start_1day
0,0,2022-09-30 23:50:00,231.68,217.70,24588842.0,189.42,238.72,GAZP,0
1,1,2022-10-03 23:50:00,219.00,215.83,5070201.0,213.63,220.55,GAZP,0
2,2,2022-10-04 23:50:00,216.48,210.72,4975859.0,208.80,216.70,GAZP,0
3,3,2022-10-05 23:50:00,211.50,209.05,5322122.0,202.85,211.50,GAZP,0
4,4,2022-10-06 23:50:00,210.00,212.86,5165564.0,209.06,216.88,GAZP,0


In [33]:
def calculate_bollinger_bands(data, window):
    """Calculate Bollinger Bands"""
    rolling_mean = data.rolling(window=window, min_periods=1).mean().values
    rolling_std = data.rolling(window=window, min_periods=1).std().values
    #upper_band = rolling_mean + (rolling_std * num_of_std)
    #lower_band = rolling_mean - (rolling_std * num_of_std)

    #сделать 3 ыев с окнами
    
    return rolling_mean, rolling_std

def calculate_rsi(data, window):
    """Calculate Relative Strength Index"""
    delta = data.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.rolling(window=window, min_periods=1).mean()
    avg_loss = loss.rolling(window=window, min_periods=1).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))

    mask = avg_loss == 0
    rsi[mask] = 100
    
    return rsi.values

def calculate_roc(data, periods):
    """Calculate Rate of Change."""
    roc = ((data - data.shift(periods)) / data.shift(periods))
    return roc.values



def calc_stats(data, window=None, feat_name=None):
    #mean, std
    rolling_mean, rolling_std = calculate_bollinger_bands(data, window)

    #min, max
    rolling_min = data.rolling(window=window, min_periods=1).min().values
    rolling_max = data.rolling(window=window, min_periods=1).max().values
    
    #rsi
    rsi = calculate_rsi(data, window)
    
    #roc
    roc = calculate_roc(data, window)
    diff = data.diff(window).values

    # #diff 1
    # diff = data.diff(1).values
    # percent_change = data.pct_change().values



    return pd.DataFrame({f'{feat_name}_ma' : rolling_mean,
                        f'{feat_name}_std' : rolling_std,
                        f'{feat_name}_min' : rolling_min,
                        f'{feat_name}_max' : rolling_max,
                        f'{feat_name}_rsi' : rsi,
                        f'{feat_name}_roc' : roc,
                        f'{feat_name}_diff' : diff,
                        })
def calc_stats_diff_1(data, feat_name=None):
    return pd.DataFrame({f'{feat_name}_roc' : data.pct_change(periods=1).values,
                        f'{feat_name}_diff' : data.diff(1).values,
                        })

#### 1min

In [39]:
dfs = []
for ticker in tqdm(df['ticker'].unique()):
    mask = np.array(df['ticker'] == ticker)
    df_ticker = df.loc[mask].copy().reset_index()

    df_close_w1 = calc_stats_diff_1(df_ticker['close'], feat_name='close_w1')
    df_volume_w1 = calc_stats_diff_1(df_ticker['volume'], feat_name='volume_w1')

    df_close_w5 = calc_stats(df_ticker['close'], window=5, feat_name='close_w5')
    df_volume_w5 = calc_stats(df_ticker['volume'], window=5, feat_name='volume_w5')
    
    df_close_w30 = calc_stats(df_ticker['close'], window=60, feat_name='close_w30')
    df_volume_w30 = calc_stats(df_ticker['volume'], window=60, feat_name='volume_w30')

    # df_close_w180 = calc_stats(df_ticker['close'], window=180, feat_name='close_w180')
    # df_volume_w180 = calc_stats(df_ticker['volume'], window=180, feat_name='volume_w180')
    
    dfs += [pd.concat([df_ticker,
                       df_close_w1, df_volume_w1,
                       df_close_w5, df_volume_w5,
                       df_close_w30, df_volume_w30,
                       # df_close_w180, df_volume_w180
                      ], axis=1)]

    assert (df_ticker.shape[0] == df_close_w1.shape[0]) and (df_ticker.shape[0] == df_close_w5.shape[0]) and (df_ticker.shape[0] == df_close_w30.shape[0]), 'error1'
    assert (df_ticker.shape[0] == dfs[-1].shape[0]) and (dfs[-1].shape[1] == (df_ticker.shape[1]+2*df_close_w1.shape[1]+2*df_close_w5.shape[1]+2*df_close_w30.shape[1])), 'error2'


df_fe = pd.concat(dfs).set_index('index')



100%|█████████████████████████████████████████████████████████████████████████| 19/19 [00:10<00:00,  1.89it/s]


In [40]:
df_fe.head()

,time,open,close,volume,low,high,ticker,index_1min_start,index_5min,open_5min,...,close_w30_rsi,close_w30_roc,close_w30_diff,volume_w30_ma,volume_w30_std,volume_w30_min,volume_w30_max,volume_w30_rsi,volume_w30_roc,volume_w30_diff
index,,,,,,,,,,,,,,,,,,,,,
0,2022-10-03 10:01:00,219.00,219.93,87793.0,218.00,220.25,GAZP,0,NaN,NaN,...,NaN,NaN,NaN,87793.000000,NaN,87793.0,87793.0,NaN,NaN,NaN
1,2022-10-03 10:02:00,219.80,220.08,61549.0,219.07,220.55,GAZP,0,NaN,NaN,...,100.000000,NaN,NaN,74671.000000,18557.310365,61549.0,87793.0,0.000000,NaN,NaN
2,2022-10-03 10:03:00,220.24,219.34,52640.0,219.02,220.38,GAZP,0,NaN,NaN,...,16.853933,NaN,NaN,67327.333333,18274.988491,52640.0,87793.0,0.000000,NaN,NaN
3,2022-10-03 10:04:00,219.17,218.96,61506.0,217.70,219.40,GAZP,0,NaN,NaN,...,11.811024,NaN,NaN,65872.000000,15202.700966,52640.0,87793.0,20.141303,NaN,NaN
4,2022-10-03 10:05:00,218.82,218.30,51464.0,217.76,218.96,GAZP,0,0.0,219.0,...,7.772021,NaN,NaN,62990.400000,14658.092656,51464.0,87793.0,16.399993,NaN,NaN


In [41]:
(df_fe['close'] == df['close']).all()

True

#### 5min

In [45]:
dfs = []
for ticker in tqdm(df_5min['ticker'].unique()):
    mask = np.array(df_5min['ticker'] == ticker)
    df_ticker = df_5min.loc[mask].copy().reset_index()

    df_close_w1 = calc_stats_diff_1(df_ticker['close_5min'], feat_name='close_5min_w1')
    df_volume_w1 = calc_stats_diff_1(df_ticker['volume_5min'], feat_name='volume_5min_w1')

    df_close_w6 = calc_stats(df_ticker['close_5min'], window=6, feat_name='close_5min_w6')
    df_volume_w6 = calc_stats(df_ticker['volume_5min'], window=6, feat_name='volume_5min_w6')
    
    df_close_w12 = calc_stats(df_ticker['close_5min'], window=12, feat_name='close_5min_w12')
    df_volume_w12 = calc_stats(df_ticker['volume_5min'], window=12, feat_name='volume_5min_w12')

    # df_close_w168 = calc_stats(df_ticker['close_5min'], window=12*14, feat_name='close_5min_w168')
    # df_volume_w168 = calc_stats(df_ticker['volume_5min'], window=12*14, feat_name='volume_5min_w168')

    
    
    dfs += [pd.concat([df_ticker,
                       df_close_w1, df_volume_w1,
                       df_close_w6, df_volume_w6,
                       df_close_w12, df_volume_w12,
                       # df_close_w168, df_volume_w168
                      ], axis=1)]

    assert (df_ticker.shape[0] == df_close_w1.shape[0]) and (df_ticker.shape[0] == df_close_w6.shape[0]) and (df_ticker.shape[0] == df_close_w12.shape[0]), 'error1'
    assert (df_ticker.shape[0] == dfs[-1].shape[0]) and (dfs[-1].shape[1] == (df_ticker.shape[1]+2*df_close_w1.shape[1]+2*df_close_w6.shape[1]+2*df_close_w12.shape[1])), 'error2'



df_5min_fe = pd.concat(dfs).set_index('index')

100%|█████████████████████████████████████████████████████████████████████████| 19/19 [00:01<00:00, 10.62it/s]


In [47]:
df_5min_fe.head()

,index_5min,time,open_5min,close_5min,volume_5min,low_5min,high_5min,ticker,index_start_5min,close_5min_w1_roc,...,close_5min_w12_rsi,close_5min_w12_roc,close_5min_w12_diff,volume_5min_w12_ma,volume_5min_w12_std,volume_5min_w12_min,volume_5min_w12_max,volume_5min_w12_rsi,volume_5min_w12_roc,volume_5min_w12_diff
index,,,,,,,,,,,,,,,,,,,,,
0,0,2022-10-03 10:05:00,219.00,218.30,314952.0,217.70,220.55,GAZP,0,NaN,...,NaN,NaN,NaN,314952.000000,NaN,314952.0,314952.0,NaN,NaN,NaN
1,1,2022-10-03 10:10:00,218.30,218.05,182525.0,216.19,218.78,GAZP,0,-0.001145,...,0.000000,NaN,NaN,248738.500000,93640.029712,182525.0,314952.0,0.000000,NaN,NaN
2,2,2022-10-03 10:15:00,218.11,215.86,193758.0,215.31,218.13,GAZP,0,-0.010044,...,0.000000,NaN,NaN,230411.666667,73429.190669,182525.0,314952.0,7.819156,NaN,NaN
3,3,2022-10-03 10:20:00,215.83,216.85,91747.0,215.70,217.39,GAZP,0,0.004586,...,28.862974,NaN,NaN,195745.500000,91659.895667,91747.0,314952.0,4.572375,NaN,NaN
4,4,2022-10-03 10:25:00,216.85,216.58,52824.0,216.40,217.16,GAZP,0,-0.001245,...,26.756757,NaN,NaN,167161.200000,101913.999959,52824.0,314952.0,3.947026,NaN,NaN


In [48]:
(df_5min_fe['close_5min'] == df_5min['close_5min']).all(), (df_5min_fe.index == df_5min_fe['index_5min']).all()

(True, True)

#### 1hour

In [51]:
dfs = []
for ticker in tqdm(df_1hour['ticker'].unique()):
    mask = np.array(df_1hour['ticker'] == ticker)
    df_ticker = df_1hour.loc[mask].copy().reset_index()

    df_close_w1 = calc_stats_diff_1(df_ticker['close_1hour'], feat_name='close_1hour_w1')
    df_volume_w1 = calc_stats_diff_1(df_ticker['volume_1hour'], feat_name='volume_1hour_w1')

    df_close_w3 = calc_stats(df_ticker['close_1hour'], window=3, feat_name='close_1hour_w3')
    df_volume_w3 = calc_stats(df_ticker['volume_1hour'], window=3, feat_name='volume_1hour_w3')
    
    df_close_w14 = calc_stats(df_ticker['close_1hour'], window=14, feat_name='close_1hour_w14')
    df_volume_w14 = calc_stats(df_ticker['volume_1hour'], window=14, feat_name='volume_1hour_w14')

    # df_close_w70 = calc_stats(df_ticker['close_1hour'], window=14*5, feat_name='close_1hour_w70')
    # df_volume_w70 = calc_stats(df_ticker['volume_1hour'], window=14*5, feat_name='volume_1hour_w70')

    
    
    dfs += [pd.concat([df_ticker,
                       df_close_w1, df_volume_w1,
                       df_close_w3, df_volume_w3,
                       df_close_w14, df_volume_w14,
                       # df_close_w70, df_volume_w70
                      ], axis=1)]
    assert (df_ticker.shape[0] == df_close_w1.shape[0]) and (df_ticker.shape[0] == df_close_w3.shape[0]) and (df_ticker.shape[0] == df_close_w14.shape[0]), 'error1'
    assert (df_ticker.shape[0] == dfs[-1].shape[0]) and (dfs[-1].shape[1] == (df_ticker.shape[1]+2*df_close_w1.shape[1]+2*df_close_w3.shape[1]+2*df_close_w14.shape[1])), 'error2'



df_1hour_fe = pd.concat(dfs).set_index('index')


100%|█████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 86.12it/s]


In [52]:
df_1hour_fe.head()

,index_1hour,time,open_1hour,close_1hour,volume_1hour,low_1hour,high_1hour,ticker,index_start_1hour,close_1hour_w1_roc,...,close_1hour_w14_rsi,close_1hour_w14_roc,close_1hour_w14_diff,volume_1hour_w14_ma,volume_1hour_w14_std,volume_1hour_w14_min,volume_1hour_w14_max,volume_1hour_w14_rsi,volume_1hour_w14_roc,volume_1hour_w14_diff
index,,,,,,,,,,,,,,,,,,,,,
0,0,2022-10-03 11:00:00,219.00,215.08,1623411.0,213.63,220.55,GAZP,0,NaN,...,NaN,NaN,NaN,1623411.0,NaN,1623411.0,1623411.0,NaN,NaN,NaN
1,1,2022-10-03 12:00:00,215.08,216.36,411786.0,214.96,216.78,GAZP,0,0.005951,...,100.000000,NaN,NaN,1017598.5,856748.253755,411786.0,1623411.0,0.000000,NaN,NaN
2,2,2022-10-03 13:00:00,216.34,215.88,237084.0,215.43,216.43,GAZP,0,-0.002219,...,72.727273,NaN,NaN,757427.0,755034.047837,237084.0,1623411.0,0.000000,NaN,NaN
3,3,2022-10-03 14:00:00,215.85,217.10,573509.0,215.53,217.77,GAZP,0,0.005651,...,83.892617,NaN,NaN,711447.5,623303.617784,237084.0,1623411.0,19.528348,NaN,NaN
4,4,2022-10-03 15:00:00,217.12,216.94,364609.0,216.50,217.49,GAZP,0,-0.000737,...,79.617834,NaN,NaN,642079.8,561640.400055,237084.0,1623411.0,17.416439,NaN,NaN


In [53]:
(df_1hour_fe['close_1hour'] == df_1hour['close_1hour']).all(), (df_1hour_fe.index == df_1hour_fe['index_1hour']).all()

(True, True)

#### 1day

In [59]:
dfs = []
for ticker in tqdm(df_1day['ticker'].unique()):
    mask = np.array(df_1day['ticker'] == ticker)
    df_ticker = df_1day.loc[mask].copy().reset_index()

    df_close_w1 = calc_stats_diff_1(df_ticker['close_1day'], feat_name='close_1day_w1')
    df_volume_w1 = calc_stats_diff_1(df_ticker['volume_1day'], feat_name='volume_1day_w1')

    df_close_w2 = calc_stats(df_ticker['close_1day'], window=2, feat_name='close_1day_w2')
    df_volume_w2 = calc_stats(df_ticker['volume_1day'], window=2, feat_name='volume_1day_w2')
    
    df_close_w5 = calc_stats(df_ticker['close_1day'], window=5, feat_name='close_1day_w5')
    df_volume_w5 = calc_stats(df_ticker['volume_1day'], window=5, feat_name='volume_1day_w5')

    df_close_w20 = calc_stats(df_ticker['close_1day'], window=5*4, feat_name='close_1day_w20')
    df_volume_w20 = calc_stats(df_ticker['volume_1day'], window=5*4, feat_name='volume_1day_w20')

    df_close_w200 = calc_stats(df_ticker['close_1day'], window=200, feat_name='close_1day_w200')
    df_volume_w200 = calc_stats(df_ticker['volume_1day'], window=200, feat_name='volume_1day_w200')

    
    
    dfs += [pd.concat([df_ticker,
                       df_close_w1, df_volume_w1,
                       df_close_w2, df_volume_w2,
                       df_close_w5, df_volume_w5,
                       df_close_w20, df_volume_w20,
                       df_close_w200, df_volume_w200], axis=1)]
    
    assert (df_ticker.shape[0] == df_close_w1.shape[0]) and (df_ticker.shape[0] == df_close_w2.shape[0]) and (df_ticker.shape[0] == df_close_w5.shape[0]) and (df_ticker.shape[0] == df_close_w20.shape[0]) and (df_ticker.shape[0] == df_close_w20.shape[0]), 'error1'
    assert (df_ticker.shape[0] == dfs[-1].shape[0]) and (dfs[-1].shape[1] == (df_ticker.shape[1]+2*df_close_w1.shape[1]+2*df_close_w2.shape[1]+2*df_close_w5.shape[1]+2*df_close_w20.shape[1]+2*df_close_w200.shape[1])), 'error2'



df_1day_fe = pd.concat(dfs).set_index('index')


100%|████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 147.99it/s]


In [61]:
df_1day_fe.head()

,index_1day,time,open_1day,close_1day,volume_1day,low_1day,high_1day,ticker,index_start_1day,close_1day_w1_roc,...,close_1day_w200_rsi,close_1day_w200_roc,close_1day_w200_diff,volume_1day_w200_ma,volume_1day_w200_std,volume_1day_w200_min,volume_1day_w200_max,volume_1day_w200_rsi,volume_1day_w200_roc,volume_1day_w200_diff
index,,,,,,,,,,,,,,,,,,,,,
0,0,2022-09-30 23:50:00,231.68,217.70,24588842.0,189.42,238.72,GAZP,0,NaN,...,NaN,NaN,NaN,2.458884e+07,NaN,24588842.0,24588842.0,NaN,NaN,NaN
1,1,2022-10-03 23:50:00,219.00,215.83,5070201.0,213.63,220.55,GAZP,0,-0.008590,...,0.000000,NaN,NaN,1.482952e+07,1.380176e+07,5070201.0,24588842.0,0.000000,NaN,NaN
2,2,2022-10-04 23:50:00,216.48,210.72,4975859.0,208.80,216.70,GAZP,0,-0.023676,...,0.000000,NaN,NaN,1.154497e+07,1.129643e+07,4975859.0,24588842.0,0.000000,NaN,NaN
3,3,2022-10-05 23:50:00,211.50,209.05,5322122.0,202.85,211.50,GAZP,0,-0.007925,...,0.000000,NaN,NaN,9.989256e+06,9.734155e+06,4975859.0,24588842.0,1.734850,NaN,NaN
4,4,2022-10-06 23:50:00,210.00,212.86,5165564.0,209.06,216.88,GAZP,0,0.018225,...,30.577849,NaN,NaN,9.024518e+06,8.701662e+06,4975859.0,24588842.0,1.721348,NaN,NaN


In [63]:
(df_1day_fe['close_1day'] == df_1day['close_1day']).all(), (df_1day_fe.index == df_1day_fe['index_1day']).all()

(True, True)

### time features

#### 1hour

In [70]:
df_1hour_fe.iloc[:15]

,index_1hour,time,open_1hour,close_1hour,volume_1hour,low_1hour,high_1hour,ticker,index_start_1hour,close_1hour_w1_roc,...,close_1hour_w14_rsi,close_1hour_w14_roc,close_1hour_w14_diff,volume_1hour_w14_ma,volume_1hour_w14_std,volume_1hour_w14_min,volume_1hour_w14_max,volume_1hour_w14_rsi,volume_1hour_w14_roc,volume_1hour_w14_diff
index,,,,,,,,,,,,,,,,,,,,,
0,0,2022-10-03 11:00:00,219.00,215.08,1623411.0,213.63,220.55,GAZP,0,NaN,...,NaN,NaN,NaN,1.623411e+06,NaN,1623411.0,1623411.0,NaN,NaN,NaN
1,1,2022-10-03 12:00:00,215.08,216.36,411786.0,214.96,216.78,GAZP,0,0.005951,...,100.000000,NaN,NaN,1.017598e+06,856748.253755,411786.0,1623411.0,0.000000,NaN,NaN
2,2,2022-10-03 13:00:00,216.34,215.88,237084.0,215.43,216.43,GAZP,0,-0.002219,...,72.727273,NaN,NaN,7.574270e+05,755034.047837,237084.0,1623411.0,0.000000,NaN,NaN
3,3,2022-10-03 14:00:00,215.85,217.10,573509.0,215.53,217.77,GAZP,0,0.005651,...,83.892617,NaN,NaN,7.114475e+05,623303.617784,237084.0,1623411.0,19.528348,NaN,NaN
4,4,2022-10-03 15:00:00,217.12,216.94,364609.0,216.50,217.49,GAZP,0,-0.000737,...,79.617834,NaN,NaN,6.420798e+05,561640.400055,237084.0,1623411.0,17.416439,NaN,NaN
5,5,2022-10-03 16:00:00,216.94,216.54,337495.0,216.00,217.25,GAZP,0,-0.001844,...,70.621469,NaN,NaN,5.913157e+05,517507.424659,237084.0,1623411.0,17.175354,NaN,NaN
6,6,2022-10-03 17:00:00,216.56,215.91,333448.0,215.81,216.68,GAZP,0,-0.002909,...,59.952038,NaN,NaN,5.544774e+05,482366.737365,237084.0,1623411.0,17.139942,NaN,NaN
7,7,2022-10-03 18:00:00,215.91,215.30,556993.0,214.50,215.98,GAZP,0,-0.002825,...,52.301255,NaN,NaN,5.547919e+05,446585.706537,237084.0,1623411.0,25.611999,NaN,NaN
8,8,2022-10-03 18:50:00,215.30,215.13,192518.0,214.87,215.30,GAZP,0,-0.000790,...,50.505051,NaN,NaN,5.145392e+05,434846.442579,192518.0,1623411.0,21.952437,NaN,NaN


In [71]:
time_cyclic = (df_1hour_fe['time'] - pd.to_datetime(df_1hour_fe['time'].dt.date) - pd.Timedelta('11:00:00')) / pd.Timedelta('12:50:00')
df_1hour_fe['sin_time_hour'] = np.sin(time_cyclic * 2 * np.pi)
df_1hour_fe['cos_time_hour'] = np.cos(time_cyclic * 2 * np.pi)

df_1hour_fe

,index_1hour,time,open_1hour,close_1hour,volume_1hour,low_1hour,high_1hour,ticker,index_start_1hour,close_1hour_w1_roc,...,close_1hour_w14_diff,volume_1hour_w14_ma,volume_1hour_w14_std,volume_1hour_w14_min,volume_1hour_w14_max,volume_1hour_w14_rsi,volume_1hour_w14_roc,volume_1hour_w14_diff,sin_time_hour,cos_time_hour
index,,,,,,,,,,,,,,,,,,,,,
0,0,2022-10-03 11:00:00,219.000,215.080,1623411.0,213.630,220.550,GAZP,0,NaN,...,NaN,1.623411e+06,NaN,1623411.0,1623411.0,NaN,NaN,NaN,0.000000e+00,1.000000
1,1,2022-10-03 12:00:00,215.080,216.360,411786.0,214.960,216.780,GAZP,0,0.005951,...,NaN,1.017598e+06,856748.253755,411786.0,1623411.0,0.000000,NaN,NaN,4.702719e-01,0.882522
2,2,2022-10-03 13:00:00,216.340,215.880,237084.0,215.430,216.430,GAZP,0,-0.002219,...,NaN,7.574270e+05,755034.047837,237084.0,1623411.0,0.000000,NaN,NaN,8.300502e-01,0.557689
3,3,2022-10-03 14:00:00,215.850,217.100,573509.0,215.530,217.770,GAZP,0,0.005651,...,NaN,7.114475e+05,623303.617784,237084.0,1623411.0,19.528348,NaN,NaN,9.948025e-01,0.101823
4,4,2022-10-03 15:00:00,217.120,216.940,364609.0,216.500,217.490,GAZP,0,-0.000737,...,NaN,6.420798e+05,561640.400055,237084.0,1623411.0,17.416439,NaN,NaN,9.258192e-01,-0.377967
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152885,152885,2024-12-25 20:00:00,32.780,32.710,62816.0,32.650,32.845,RUAL,144800,0.001991,...,0.230,1.581544e+05,112941.432886,33202.0,455628.0,50.318124,0.119475,6704.0,-9.535464e-01,-0.301246
152886,152886,2024-12-25 21:00:00,32.725,32.660,21135.0,32.640,32.745,RUAL,144800,-0.001529,...,0.235,1.517569e+05,118248.229956,21135.0,455628.0,45.697172,-0.809079,-89565.0,-9.831929e-01,0.182570
152887,152887,2024-12-25 22:00:00,32.660,32.670,62547.0,32.465,32.675,RUAL,144800,0.000306,...,0.300,1.538529e+05,116227.894767,21135.0,455628.0,51.460411,0.883832,29345.0,-7.818315e-01,0.623490


#### 1day

In [77]:
day_of_week_cyclic = np.minimum(df_1day_fe['time'].dt.dayofweek, 4) / 4
df_1day_fe['sin_time_weekday'] = np.sin(day_of_week_cyclic * 2 * np.pi)
df_1day_fe['cos_time_weekday'] = np.cos(day_of_week_cyclic * 2 * np.pi)

df_1day_fe.head()

,index_1day,time,open_1day,close_1day,volume_1day,low_1day,high_1day,ticker,index_start_1day,close_1day_w1_roc,...,close_1day_w200_diff,volume_1day_w200_ma,volume_1day_w200_std,volume_1day_w200_min,volume_1day_w200_max,volume_1day_w200_rsi,volume_1day_w200_roc,volume_1day_w200_diff,sin_time_weekday,cos_time_weekday
index,,,,,,,,,,,,,,,,,,,,,
0,0,2022-09-30 23:50:00,231.68,217.70,24588842.0,189.42,238.72,GAZP,0,NaN,...,NaN,2.458884e+07,NaN,24588842.0,24588842.0,NaN,NaN,NaN,-2.449294e-16,1.000000e+00
1,1,2022-10-03 23:50:00,219.00,215.83,5070201.0,213.63,220.55,GAZP,0,-0.008590,...,NaN,1.482952e+07,1.380176e+07,5070201.0,24588842.0,0.000000,NaN,NaN,0.000000e+00,1.000000e+00
2,2,2022-10-04 23:50:00,216.48,210.72,4975859.0,208.80,216.70,GAZP,0,-0.023676,...,NaN,1.154497e+07,1.129643e+07,4975859.0,24588842.0,0.000000,NaN,NaN,1.000000e+00,6.123234e-17
3,3,2022-10-05 23:50:00,211.50,209.05,5322122.0,202.85,211.50,GAZP,0,-0.007925,...,NaN,9.989256e+06,9.734155e+06,4975859.0,24588842.0,1.734850,NaN,NaN,1.224647e-16,-1.000000e+00
4,4,2022-10-06 23:50:00,210.00,212.86,5165564.0,209.06,216.88,GAZP,0,0.018225,...,NaN,9.024518e+06,8.701662e+06,4975859.0,24588842.0,1.721348,NaN,NaN,-1.000000e+00,-1.836970e-16


## 2.6 Save data

In [80]:
df_fe.columns

Index(['time', 'open', 'close', 'volume', 'low', 'high', 'ticker',
       'index_1min_start', 'index_5min', 'open_5min', 'close_5min',
       'volume_5min', 'low_5min', 'high_5min', 'index_start_5min',
       'index_1hour', 'open_1hour', 'close_1hour', 'volume_1hour', 'low_1hour',
       'high_1hour', 'index_start_1hour', 'index_1day', 'open_1day',
       'close_1day', 'volume_1day', 'low_1day', 'high_1day',
       'index_start_1day', 'result', 'delta_time', 'res_price', 'res_ind',
       'close_w1_roc', 'close_w1_diff', 'volume_w1_roc', 'volume_w1_diff',
       'close_w5_ma', 'close_w5_std', 'close_w5_min', 'close_w5_max',
       'close_w5_rsi', 'close_w5_roc', 'close_w5_diff', 'volume_w5_ma',
       'volume_w5_std', 'volume_w5_min', 'volume_w5_max', 'volume_w5_rsi',
       'volume_w5_roc', 'volume_w5_diff', 'close_w30_ma', 'close_w30_std',
       'close_w30_min', 'close_w30_max', 'close_w30_rsi', 'close_w30_roc',
       'close_w30_diff', 'volume_w30_ma', 'volume_w30_std', 'volume_w30

In [82]:
!mkdir experiments/feat_engin/data

mkdir: experiments/feat_engin/data: File exists


In [84]:
cols = df_fe.columns[~df_fe.columns.isin(['open', 'low', 'high', 
                    'open_5min', 'close_5min', 'volume_5min', 'low_5min', 'high_5min', 
                   'open_1hour', 'close_1hour', 'volume_1hour', 'low_1hour', 'high_1hour',
                    'open_1day', 'close_1day', 'volume_1day', 'low_1day', 'high_1day'
                   ])]

for col in tqdm(cols):
    try:
        df_fe[col] = df_fe[col].astype(np.float32)
    except:
        print(col)

#save
dump_pkl(df_fe[cols], 'experiments/feat_engin/data/data_shift_time_1min.pkl')

  0%|                                                                                  | 0/47 [00:00<?, ?it/s]

time
ticker


 43%|███████████████████████████████                                          | 20/47 [00:00<00:00, 32.86it/s]

result
delta_time


100%|█████████████████████████████████████████████████████████████████████████| 47/47 [00:01<00:00, 40.64it/s]


In [88]:
(~df_fe.columns.isin(['ticker', 'result'])).sum(), df_fe[df_fe.columns[~df_fe.columns.isin(['ticker', 'result'])]].mean().nunique()


(63, 63)

In [90]:
#5min

In [91]:
df_5min_fe.columns

Index(['index_5min', 'time', 'open_5min', 'close_5min', 'volume_5min',
       'low_5min', 'high_5min', 'ticker', 'index_start_5min',
       'close_5min_w1_roc', 'close_5min_w1_diff', 'volume_5min_w1_roc',
       'volume_5min_w1_diff', 'close_5min_w6_ma', 'close_5min_w6_std',
       'close_5min_w6_min', 'close_5min_w6_max', 'close_5min_w6_rsi',
       'close_5min_w6_roc', 'close_5min_w6_diff', 'volume_5min_w6_ma',
       'volume_5min_w6_std', 'volume_5min_w6_min', 'volume_5min_w6_max',
       'volume_5min_w6_rsi', 'volume_5min_w6_roc', 'volume_5min_w6_diff',
       'close_5min_w12_ma', 'close_5min_w12_std', 'close_5min_w12_min',
       'close_5min_w12_max', 'close_5min_w12_rsi', 'close_5min_w12_roc',
       'close_5min_w12_diff', 'volume_5min_w12_ma', 'volume_5min_w12_std',
       'volume_5min_w12_min', 'volume_5min_w12_max', 'volume_5min_w12_rsi',
       'volume_5min_w12_roc', 'volume_5min_w12_diff'],
      dtype='object')

In [94]:
cols = df_5min_fe.columns[~df_5min_fe.columns.isin(['index_5min', 'open_5min', 'low_5min', 'high_5min', 'index_start_5min'
                   ])]

for col in tqdm(cols):
    try:
        df_5min_fe[col] = df_5min_fe[col].astype(np.float32)
    except:
        print(col)

dump_pkl(df_5min_fe[cols], 'experiments/feat_engin/data/data_shift_time_5min.pkl')

100%|████████████████████████████████████████████████████████████████████████| 36/36 [00:00<00:00, 373.78it/s]


time
ticker


In [96]:
(~df_5min_fe.columns.isin(['ticker', 'result'])).sum(), df_5min_fe[df_5min_fe.columns[~df_5min_fe.columns.isin(['ticker', 'result'])]].mean().nunique()


(40, 40)

In [98]:
#1hour

In [99]:
df_1hour_fe.columns

Index(['index_1hour', 'time', 'open_1hour', 'close_1hour', 'volume_1hour',
       'low_1hour', 'high_1hour', 'ticker', 'index_start_1hour',
       'close_1hour_w1_roc', 'close_1hour_w1_diff', 'volume_1hour_w1_roc',
       'volume_1hour_w1_diff', 'close_1hour_w3_ma', 'close_1hour_w3_std',
       'close_1hour_w3_min', 'close_1hour_w3_max', 'close_1hour_w3_rsi',
       'close_1hour_w3_roc', 'close_1hour_w3_diff', 'volume_1hour_w3_ma',
       'volume_1hour_w3_std', 'volume_1hour_w3_min', 'volume_1hour_w3_max',
       'volume_1hour_w3_rsi', 'volume_1hour_w3_roc', 'volume_1hour_w3_diff',
       'close_1hour_w14_ma', 'close_1hour_w14_std', 'close_1hour_w14_min',
       'close_1hour_w14_max', 'close_1hour_w14_rsi', 'close_1hour_w14_roc',
       'close_1hour_w14_diff', 'volume_1hour_w14_ma', 'volume_1hour_w14_std',
       'volume_1hour_w14_min', 'volume_1hour_w14_max', 'volume_1hour_w14_rsi',
       'volume_1hour_w14_roc', 'volume_1hour_w14_diff', 'sin_time_hour',
       'cos_time_hour'],
     

In [101]:
cols = df_1hour_fe.columns[~df_1hour_fe.columns.isin(['index_1hour', 'open_1hour', 'low_1hour', 'high_1hour', 'index_start_1hour'
                   ])]

for col in tqdm(cols):
    try:
        df_1hour_fe[col] = df_1hour_fe[col].astype(np.float32)
    except:
        print(col)

dump_pkl(df_1hour_fe[cols], 'experiments/feat_engin/data/data_shift_time_1hour.pkl')

100%|███████████████████████████████████████████████████████████████████████| 38/38 [00:00<00:00, 5154.71it/s]

time
ticker


In [103]:
(~df_1hour_fe.columns.isin(['ticker', 'result'])).sum(), df_1hour_fe[df_1hour_fe.columns[~df_1hour_fe.columns.isin(['ticker', 'result'])]].mean().nunique()


(42, 42)

In [105]:
#1day

In [107]:
df_1day_fe.columns

Index(['index_1day', 'time', 'open_1day', 'close_1day', 'volume_1day',
       'low_1day', 'high_1day', 'ticker', 'index_start_1day',
       'close_1day_w1_roc', 'close_1day_w1_diff', 'volume_1day_w1_roc',
       'volume_1day_w1_diff', 'close_1day_w2_ma', 'close_1day_w2_std',
       'close_1day_w2_min', 'close_1day_w2_max', 'close_1day_w2_rsi',
       'close_1day_w2_roc', 'close_1day_w2_diff', 'volume_1day_w2_ma',
       'volume_1day_w2_std', 'volume_1day_w2_min', 'volume_1day_w2_max',
       'volume_1day_w2_rsi', 'volume_1day_w2_roc', 'volume_1day_w2_diff',
       'close_1day_w5_ma', 'close_1day_w5_std', 'close_1day_w5_min',
       'close_1day_w5_max', 'close_1day_w5_rsi', 'close_1day_w5_roc',
       'close_1day_w5_diff', 'volume_1day_w5_ma', 'volume_1day_w5_std',
       'volume_1day_w5_min', 'volume_1day_w5_max', 'volume_1day_w5_rsi',
       'volume_1day_w5_roc', 'volume_1day_w5_diff', 'close_1day_w20_ma',
       'close_1day_w20_std', 'close_1day_w20_min', 'close_1day_w20_max',
      

In [108]:
cols = df_1day_fe.columns[~df_1day_fe.columns.isin(['index_1day', 'open_1day', 'low_1day', 'high_1day', 'index_start_1day',
                                                    'close_1day_w200_roc', 'close_1day_w200_diff',
                                                    'volume_1day_w200_roc', 'volume_1day_w200_diff',
                   ])]

for col in tqdm(cols):
    try:
        df_1day_fe[col] = df_1day_fe[col].astype(np.float32)
    except:
        print(col)

dump_pkl(df_1day_fe[cols], 'experiments/feat_engin/data/data_shift_time_1day.pkl')

100%|██████████████████████████████████████████████████████████████████████| 62/62 [00:00<00:00, 13635.72it/s]

time
ticker


In [110]:
(~df_1day_fe.columns.isin(['ticker', 'result'])).sum(), df_1day_fe[df_1day_fe.columns[~df_1day_fe.columns.isin(['ticker', 'result'])]].mean().nunique()

(70, 70)

In [112]:
cols

Index(['time', 'close_1day', 'volume_1day', 'ticker', 'close_1day_w1_roc',
       'close_1day_w1_diff', 'volume_1day_w1_roc', 'volume_1day_w1_diff',
       'close_1day_w2_ma', 'close_1day_w2_std', 'close_1day_w2_min',
       'close_1day_w2_max', 'close_1day_w2_rsi', 'close_1day_w2_roc',
       'close_1day_w2_diff', 'volume_1day_w2_ma', 'volume_1day_w2_std',
       'volume_1day_w2_min', 'volume_1day_w2_max', 'volume_1day_w2_rsi',
       'volume_1day_w2_roc', 'volume_1day_w2_diff', 'close_1day_w5_ma',
       'close_1day_w5_std', 'close_1day_w5_min', 'close_1day_w5_max',
       'close_1day_w5_rsi', 'close_1day_w5_roc', 'close_1day_w5_diff',
       'volume_1day_w5_ma', 'volume_1day_w5_std', 'volume_1day_w5_min',
       'volume_1day_w5_max', 'volume_1day_w5_rsi', 'volume_1day_w5_roc',
       'volume_1day_w5_diff', 'close_1day_w20_ma', 'close_1day_w20_std',
       'close_1day_w20_min', 'close_1day_w20_max', 'close_1day_w20_rsi',
       'close_1day_w20_roc', 'close_1day_w20_diff', 'volume_1da

# TMP

In [285]:
df_fe[df_fe.columns[~df_fe.columns.isin(['ticker', 'time', 'result', 'delta_time'])]].abs().max().max()

38785459.0

In [287]:
df_5min_fe[df_5min_fe.columns[~df_5min_fe.columns.isin(['ticker', 'time'])]].abs().max().max()

5050540.0

In [289]:
df_1hour_fe[df_1hour_fe.columns[~df_1hour_fe.columns.isin(['ticker', 'time'])]].abs().max().max()

16132700.0

In [291]:
df_1day_fe[df_1day_fe.columns[~df_1day_fe.columns.isin(['ticker', 'time'])]].abs().max().max()

38785460.0

In [293]:
np.array([38785459]).astype(np.float32)

array([38785460.], dtype=float32)

In [297]:
df_1day_fe[df_1day_fe.columns[~df_1day_fe.columns.isin(['ticker', 'time'])]].max()

index_1day                    10760
open_1day                    8450.0
close_1day                   8444.0
volume_1day              38785460.0
low_1day                     8315.0
                            ...    
volume_1day_w200_rsi          100.0
volume_1day_w200_roc     189.387988
volume_1day_w200_diff    27633284.0
sin_time_weekday                1.0
cos_time_weekday                1.0
Length: 69, dtype: object